In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import product

from backtesterClass.orderClass import orders
from backtesterClass.orderBookClass import OBData
from backtesterClass.analysisClass import analysisClass

from strats.basicStrat import basicStrat
from strats.movingAverageStrat import movingAverageStrat
from strats.rsiStrat import rsiStrat
from strats.momentumStrat import momentumStrat
from strats.momentumOnlineLearn import momentumOnlineLearnStrat
from strats.LTSMOnlineLearn import LTSMOnlineLearnStrat

from scipy.stats import spearmanr, pearsonr

from cProfile import Profile
from pstats import SortKey, Stats
import gc

In [2]:
# Improve computationnal performance of the backtester - increases garbage collector threshold

allocs, gen1, gen2 = gc.get_threshold()
allocs = 5000
gen1 = gen1*2
gen2=gen2*2
gc.set_threshold(allocs, gen1,gen2)

In [3]:
db_path = r"backtesterClass/stockDB.db"

dataClass = OBData(db_path)

In [4]:
autoTrader = basicStrat("autoTrader")
movingAverageTrader = movingAverageStrat("movingAverageTrader", short_window = 20, long_window=200)
rsiTrader = rsiStrat("rsiTrader", window=50, buyThreshold=30, sellThreshold=70, alpha=0.05)
momentumTrader = momentumStrat("momentumTrader", short_window = 20, long_window=200, RSI_window=50, sellThreshold=70,buyThreshold=30, alpha=0.05)
momentumOnlineTrader = momentumOnlineLearnStrat("momentumOnlineTrader", short_window = 20, long_window=200, RSI_window=50, sellThreshold=70,buyThreshold=30, alpha=0.05 ,trainLen = 200, forecast=1)
LTMSOnlineTrader = LTSMOnlineLearnStrat("LTSMOnlineTrader", short_window = 20, long_window=200, RSI_window=50, sellThreshold=70,buyThreshold=30, alpha=0.05 ,trainLen = 200, forecast=1)

['cumulative returns', 'lag_ret_1', 'lag_ret_10', 'lag_ret_2', 'lag_ret_5', 'long_ma', 'rsi', 'short_ma'] | StandardScaler | RollingRegressor


In [5]:
# def compute_spearman():
#     preds = np.array(momentumOnlineTrader.prediction)[:,0]
#     preds = np.array([v[0]<v[1] if (v[0] > 0.95) or (v[1]>0.95) else None for v in preds], dtype=float)
#     y = np.array(momentumOnlineTrader.prediction)[:,1]
#     valid_indices = ~np.isnan(preds)
#     filtred_y = y[valid_indices].tolist()
#     filtred_preds = preds[valid_indices].tolist()
#     ic = spearmanr(filtred_preds, filtred_y)
#     return ic

In [6]:
# short_windows = [50, 100, 250, 500, 1000]
# long_windows = [500, 1000, 2500, 5000, 10000]
# RSI_windows = [50, 100, 250, 500, 1000]
# sellThresholds = [60, 65, 70] 
# buyThresholds = [40, 35, 30] 
# forecasts= [5, 10, 25, 50, 100]

# param_combinations = product(short_windows, long_windows, RSI_windows, sellThresholds, buyThresholds, forecasts)

# with open('simulation_results.csv', mode='w', newline='') as file:
#     writer = csv.writer(file)
#     # Write the header row (optional)
#     writer.writerow(["ic", "short_window", "long_window", "RSI_window", "sellThreshold", "buyThreshold", "forecast"])


#     # Iterate over each combination
#     for short_window, long_window, RSI_window, sellThreshold, buyThreshold, forecast in tqdm(param_combinations):
#         # Instantiate the momentumOnlineTrader with the current combination
#         momentumOnlineTrader = momentumOnlineLearnStrat(
#             "momentumOnlineTrader", 
#             short_window=short_window, 
#             long_window=long_window, 
#             RSI_window=RSI_window, 
#             sellThreshold=sellThreshold, 
#             buyThreshold=buyThreshold, 
#             alpha=2, 
#             forecast=forecast, 
#             trainLen=20000
#         )
#         dataClass = OBData(pd.read_csv(r'/Users/leo/Downloads/BTCUSD_241227-bookTicker-2024-09-01.csv', sep=","))

#         # Run the strategy for each step in the data
#         for _ in range(len(dataClass.OBData_)):
#             orderClass = orders(dataClass.OBData_[OBData.step])
#             momentumOnlineTrader.strategy(orderClass)
#             OBData.step += 1
        
#         ic = compute_spearman()
#         writer.writerow([ic, short_window, long_window, RSI_window, sellThreshold, buyThreshold, forecast])

#         # print(f"ic: {ic}, short_window: {short_window}, long_window: {long_window}, RSI_window: {RSI_window}, sellThreshold: {sellThreshold}, buyThreshold:{buyThreshold}, forecast:{forecast}")

In [7]:
for _ in tqdm(range(len(dataClass.OBData_))):
    orderClass = orders()
    # autoTrader.strategy(orderClass)
    movingAverageTrader.strategy(orderClass)
    rsiTrader.strategy(orderClass)
    # momentumTrader.strategy(orderClass)
    # momentumOnlineTrader.strategy(orderClass)
    # LTMSOnlineTrader.strategy(orderClass)
    OBData.step +=1 

100%|██████████| 4243/4243 [00:24<00:00, 171.29it/s]


In [8]:
# analysisBasic = analysisClass(autoTrader ,path=r"/Users/leo/Desktop/EDHEC MsC FENg/Master Project/Project/Results", dashboardName="executionBasicStrat", dbName="dbBasicStrat")
# analysisBasic.create_dashboard(asset="AAPL")
# analysisBasic.dataBase()

In [9]:
analysisMA = analysisClass(movingAverageTrader, path=r"Results", dashboardName="executionMovingAverageStrat", dbName="dbMovingAverageStrat")
analysisMA.create_dashboard(asset="AAPL")
analysisMA.dataBase()

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBA

In [10]:
analysisRSI = analysisClass(rsiTrader, path=r"Results", dashboardName="executionRSIStrat", dbName="dbRSIStrat")
analysisRSI.create_dashboard("AAPL")
analysisRSI.dataBase()

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBA

In [11]:
# analysisMomentum = analysisClass(momentumTrader, path=r"Results", dashboardName="executionMomentumStrat", dbName="dbMomentumStrat")
# analysisMomentum.create_dashboard("AAPL")
# analysisMomentum.dataBase()

In [12]:
# analysisOLMomentum = analysisClass(momentumOnlineTrader, path=r"Results", dashboardName="executionOLMomentumStrat", dbName="dbOLMomentumStrat")
# analysisOLMomentum.create_dashboard("AAPL")
# analysisOLMomentum.dataBase()

In [15]:
analysisLTSM = analysisClass(LTMSOnlineTrader, path=r"Results", dashboardName="executionLTSMStrat", dbName="dbLTSMStrat")
analysisLTSM.create_dashboard("AAPL")
analysisLTSM.dataBase()

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/Users/leo/Desktop/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBA

In [ ]:
y = np.array(LTMSOnlineTrader.prediction[250])[:,1]-1
y_pred = np.array(LTMSOnlineTrader.prediction[250])[:,0]-1
ic = pearsonr(y_pred, y)
ic

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(10,6))

# Determine which points are correct (same sign) and incorrect (opposite sign)
same_sign = np.sign(y_pred) == np.sign(y)
diff_sign = ~same_sign  # Logical NOT

# Scatter plot with different colors
plt.scatter(y_pred[same_sign], y[same_sign], color='green', label='Same sign (True)')
plt.scatter(y_pred[diff_sign], y[diff_sign], color='red', label='Different sign (False)')

# Linear regression (best fit line)
m, b = np.polyfit(y_pred, y, 1)
x_fit = np.linspace(-0.3, 0.3, 100)
y_fit = m * x_fit + b
plt.plot(x_fit, y_fit, color='darkred', label=f'Best fit: y = {m:.2f}x + {b:.2f}')

# R² calculation
y_pred_fit = m * y_pred + b
residuals = y - y_pred_fit
ss_res = np.sum(residuals**2)
ss_tot = np.sum((y - np.mean(y))**2)
r_squared = 1 - (ss_res / ss_tot)

# Annotate R² on the upper right
plt.text(0.18, 0.13, f'$R^2 = {r_squared:.3f}$', fontsize=12, color='black')

# Formatting
plt.xlim(-0.3, 0.3)
plt.ylim(-0.3, 0.3)
plt.xlabel("Predicted Return")
plt.ylabel("Actual Return")
plt.title("Predicted vs Actual Returns\n(Green: Same Sign, Red: Opposite Sign)")
plt.legend()
plt.grid(True)
plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
plt.axvline(0, color='gray', linestyle='--', linewidth=0.5)
plt.show()
